Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,roc_auc_score,f1_score
from sklearn.metrics import confusion_matrix
import re
import string

In [2]:
df_fake = pd.read_csv("/content/Fake.csv")
df_true = pd.read_csv("/content/True.csv")

In [3]:
df_fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


Inserting a column "class" and considering it as target feature

In [5]:
df_fake["class"] = 0
df_true["class"] = 1

In [6]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

Merging Both The Dataframes

In [7]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(5)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [8]:
df_merge.columns
df = df_merge.drop(["title", "subject","date"], axis = 1)

Creating a function to process the texts

In [9]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [10]:
df["text"] = df["text"].apply(wordopt)

In [11]:
x = df["text"]
y = df["class"]

Splitting the data

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

Converting text to vectors

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

Implementing Various algorithms/methods

  1.Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression
classifierlr = LogisticRegression()
classifierlr.fit(xv_train, y_train)
LR_predict = classifierlr.predict(xv_test)

In [15]:
print('ROCAUC score:',roc_auc_score(y_test,LR_predict))
print('Accuracy score:',accuracy_score(y_test, LR_predict))
print('F1 score:',f1_score(y_test,LR_predict))
print('Precision:',precision_score(y_test,LR_predict))
print('Recall:',recall_score(y_test,LR_predict))

ROCAUC score: 0.9833097822987446
Accuracy score: 0.9832962138084632
F1 score: 0.9827308312226571
Precision: 0.981751265143383
Recall: 0.9837123540258144


In [16]:
confusion_matrix(y_test,LR_predict)

array([[6843,  119],
       [ 106, 6402]])

In [17]:
print(classification_report(y_test, LR_predict))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      6962
           1       0.98      0.98      0.98      6508

    accuracy                           0.98     13470
   macro avg       0.98      0.98      0.98     13470
weighted avg       0.98      0.98      0.98     13470



2.Perceptron Learning

In [18]:
from sklearn.linear_model import Perceptron
Permodel = Perceptron()
Permodel.fit(xv_train,y_train)
Per_Pred=Permodel.predict(xv_test)

In [19]:
print('ROCAUC score:',roc_auc_score(y_test,Per_Pred))
print('Accuracy score:',accuracy_score(y_test,Per_Pred))
print('F1 score:',f1_score(y_test,Per_Pred))
print('Precision:',precision_score(y_test,Per_Pred))
print('Recall:',recall_score(y_test,Per_Pred))

ROCAUC score: 0.9822041667233151
Accuracy score: 0.9827023014105419
F1 score: 0.981832358674464
Precision: 0.9966756371695425
Recall: 0.9674247080516287


In [20]:
confusion_matrix(y_test,Per_Pred)

array([[6941,   21],
       [ 212, 6296]])

In [21]:
print(classification_report(y_test, Per_Pred))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6962
           1       1.00      0.97      0.98      6508

    accuracy                           0.98     13470
   macro avg       0.98      0.98      0.98     13470
weighted avg       0.98      0.98      0.98     13470



3.Random Forest Classifier

In [22]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(xv_train, y_train)
rfc_predict = rfc.predict(xv_test)

In [23]:
print('ROCAUC score:',roc_auc_score(y_test, rfc_predict))
print('Accuracy score:',accuracy_score(y_test, rfc_predict))
print('F1 score:',f1_score(y_test, rfc_predict))
print('Precision:',precision_score(y_test,rfc_predict))
print('Recall:',recall_score(y_test,rfc_predict))

ROCAUC score: 0.9871144823942847
Accuracy score: 0.9871566443949518
F1 score: 0.9866974240676663
Precision: 0.9875327074034169
Recall: 0.9858635525507068


In [24]:
confusion_matrix(y_test,rfc_predict)

array([[6881,   81],
       [  92, 6416]])

In [25]:
print(classification_report(y_test, rfc_predict))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6962
           1       0.99      0.99      0.99      6508

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



4.XGB Classifier

In [26]:
from xgboost import XGBClassifier
import xgboost as XGB
model2=XGBClassifier()
model2.fit(xv_train,y_train)
predictions=model2.predict(xv_test)

In [27]:
print('ROCAUC score:',roc_auc_score(y_test,predictions))
print('Accuracy score:',accuracy_score(y_test,predictions))
print('F1 score:',f1_score(y_test,predictions))
print('Precision:',precision_score(y_test,predictions))
print('Recall:',recall_score(y_test,predictions))

ROCAUC score: 0.9961802696771497
Accuracy score: 0.9961395694135116
F1 score: 0.9960104342488875
Precision: 0.9946368372663194
Recall: 0.9973878303626306


In [28]:
confusion_matrix(y_test,predictions)

array([[6927,   35],
       [  17, 6491]])

In [29]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      6962
           1       0.99      1.00      1.00      6508

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



5.Decision Tree

In [30]:
from sklearn.tree import DecisionTreeClassifier 
classifier = DecisionTreeClassifier() 
classifier = classifier.fit(xv_train, y_train)  
Y_pred = classifier.predict(xv_test) 

In [31]:
print('ROCAUC score:',roc_auc_score(y_test, Y_pred))
print('Accuracy score:',accuracy_score(y_test,Y_pred ))
print('F1 score:',f1_score(y_test,Y_pred))
print('Precision:',precision_score(y_test,Y_pred))
print('Recall:',recall_score(y_test,Y_pred))

ROCAUC score: 0.9952850552132421
Accuracy score: 0.9953229398663697
F1 score: 0.9951549642390218
Precision: 0.9961508852963818
Recall: 0.994161032575292


In [32]:
confusion_matrix(y_test,Y_pred)

array([[6937,   25],
       [  38, 6470]])

In [33]:
print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6962
           1       1.00      0.99      1.00      6508

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470

